In [1]:
import pandas as pd

In [13]:
df = pd.read_csv("../data/anime.csv", encoding="utf-8")

In [ ]:
df["title_len"] = df["title"].map(len)
df.head(2)

In [15]:
df.groupby("type")["title_len"].mean().sort_values(ascending=False)

type
Special       41.894737
TV Special    39.625000
PV            36.000000
Movie         32.701220
OVA           31.829787
ONA           23.857143
TV            23.250000
CM            21.000000
Music         10.375000
Name: title_len, dtype: float64

In [ ]:
df["rating_group"] = df["rating"].apply(lambda x: {
    "R - 17+ (violence & profanity)": "adult",
    "PG - Children": "kids",
    "PG-13 - Teens 13 or older": "teens",
    "G - All Ages": "kids",
    "R+ - Mild Nudity": "adult"}.get(x, "Unknown rating"))
df

In [ ]:
df["type"] = df["type"].map(lambda x: str(x).lower() if x == "TV" or x == "Movie" or x == "OVA" else x)
df